In [1]:
from plots import just_plot
from misc import *
from nets import *
from buffer import ReplayBuffer
import timeit
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm as tqdm
tf.keras.backend.set_floatx('float32')
from collections import deque
from datetime import datetime
import random
import matplotlib
from environment import Environment


amplitude=0.4
dolinar_layers=2
number_phases=2
total_episodes = 10**3
buffer_size=500
batch_size=64
ep_guess=0.01
noise_displacement=0.5
lr_actor=0.01
lr_critic=0.001
tau=0.005


exper = np.load("example_buffer/2_sample.npy")
env = Environment(amplitude=amplitude, dolinar_layers = dolinar_layers, number_phases=number_phases)
# buffer = ReplayBuffer(buffer_size=buffer_size)

critic = Critic(nature="primary",valreg=0.01, dolinar_layers = dolinar_layers, number_phases=number_phases)
critic_target = Critic(nature="target", dolinar_layers = dolinar_layers, number_phases=number_phases)
actor = Actor(nature="primary", dolinar_layers = dolinar_layers)
actor_target = Actor(nature="target", dolinar_layers = dolinar_layers)

optimizer_critic = tf.keras.optimizers.Adam(lr=lr_critic)
optimizer_actor = tf.keras.optimizers.Adam(lr=lr_actor)

In [2]:
experiences = exper.astype(np.float32)
targeted_experience = actor_target.process_sequence_of_experiences_tf(experiences)
sequences, zeroed_rews = critic_target.process_sequence_tf(targeted_experience)
labels_critic = critic_target.give_td_errors_tf( sequences, zeroed_rews)

In [3]:
@tf.function
def critic_grad_tf(critic, experiences):
    with tf.GradientTape() as tape:
        unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
        to_stack = []
        actions_wathed_index = []
        for index in range(0,experiences.shape[-1]-3,2): # I consider from first outcome to last one (but guess)
            actions_wathed_index.append(index)
            to_stack.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))

        actions_indexed = tf.concat(to_stack,axis=1)
        tape.watch(actions_indexed)


        index_actions=0
        watched_exps=[tf.ones((experiences.shape[0],1,1))*actor.pad_value]
        watched_actions_unstacked = tf.unstack(actions_indexed, axis=1)
        for index in range(0,experiences.shape[-1]-1): 
            if index in actions_wathed_index:
                watched_exps.append(tf.expand_dims(watched_actions_unstacked[index_actions], axis=2))
                index_actions+=1
            else:
                
                watched_exps.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))

        qvals = critic(tf.reshape(tf.concat(watched_exps, axis=2), (experiences.shape[0],critic.dolinar_layers+1,2)))

        dq_da = tape.gradient(qvals, actions_indexed)
    return dq_da
dq_da = critic_grad_tf(critic, experiences)


In [4]:
context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
actor(context_outcome_actor)


<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[0.08953033]]], dtype=float32)>

In [11]:

def actor_grad_tf(actor, dq_da, experiences, optimizer_actor):
    unstacked_exp = tf.unstack(experiences, axis=1)
    actions_per_episode={}
    context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
    finns = [tf.multiply(actor(context_outcome_actor).numpy(), tf.ones((experiences.shape[0],1,1)))]

    with tf.GradientTape() as tape:
        tape.watch(actor.trainable_variables)
        for index in range(1,2*actor.dolinar_layers-2,2):
            actions_per_episode[str(index)] = []
            for k in tf.unstack(unstacked_exp[index]):
                actions_per_episode[str(index)].append(actor(tf.reshape(k, (1,1,1))))
            finns.append(tf.concat(actions_per_episode[str(index)], axis=0))
        final_preds = tf.concat(finns, axis=1)
        da_dtheta=tape.gradient(final_preds, actor.trainable_variables, output_gradients=-dq_da)
        optimizer_actor.apply_gradients(zip(da_dtheta, actor.trainable_variables))
    return

In [13]:
%timeit actor_grad_tf(actor, dq_da, experiences, optimizer_actor)

220 ms ± 5.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
tf.multiply(actor(context_outcome_actor), tf.ones((experiences.shape[0],1,1)))

<tf.Tensor: shape=(15, 1, 1), dtype=float32, numpy=
array([[[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]],

       [[-1.]]], dtype=float32)>

In [20]:
@tf.function
def actor_grad_tf(actor, dq_da, experiences, optimizer_actor):
    unstacked_exp = tf.unstack(experiences, axis=1)
    actions_per_episode={}
    context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
    finns = [tf.multiply(actor(context_outcome_actor), tf.ones((experiences.shape[0],1,1)))]

    with tf.GradientTape() as tape:
        tape.watch(actor.trainable_variables)
        for index in range(1,2*actor.dolinar_layers-2,2):
            actions_per_episode[str(index)] = []
            for k in tf.unstack(unstacked_exp[index]):
                actions_per_episode[str(index)].append(actor(tf.reshape(k, (1,1,1))))
            finns.append(tf.concat(actions_per_episode[str(index)], axis=0))
        final_preds = tf.concat(finns, axis=1)
        da_dtheta=tape.gradient(final_preds, actor.trainable_variables, output_gradients=-dq_da)
        optimizer_actor.apply_gradients(zip(da_dtheta, actor.trainable_variables))
    return

In [21]:
%timeit actor_grad_tf(actor, dq_da, experiences, optimizer_actor)

46 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
